# Setup

In [26]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

# Question 1

In [27]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    # img is the source image
    # num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
    # line_color is the color of the grid line.
    # The output of the function should be image with grids
    img_copy = img.copy()
    
    height, width = img.shape[:2]
    
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)
    
    x1, y1 = 0, 0

    for y in range(0, height, M):
        for x in range(0, width, N):
    #         if (height - y) < M or (width - x) < N:
    #             break

            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N

            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                tile = img[y:height, x:width]
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
    
    cv.imshow('patched image', img_copy)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [28]:
img = cv.imread('dog.jfif')
crop_grid(img, 2, 3, (0,255,0))

# Question2

In [29]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

for i in range(0, 15):
    alpha = i /10.0
    dst = cv.addWeighted(img1, alpha, img2, 0.25, 0)
    cv.imshow('blend_image', dst)
    cv.waitKey(100)

cv.waitKey(0)
cv.destroyAllWindows()

# Question 3

In [30]:
img = cv.imread('lena.jfif')
rows, cols = img.shape[:2]

# Center coordinate
image_center = ((cols-1)/2.0, (rows-1)/2.0)
M = cv.getRotationMatrix2D(image_center, 45, 1)

# Calculate the cos and sin, then take the absolute
abs_cos = abs(M[0,0]) 
abs_sin = abs(M[0,1])

# Calculate new width and height
new_w = int(rows * abs_sin + cols * abs_cos)
new_h = int(rows * abs_cos + cols * abs_sin)

# Calculate new center coordinate
M[0, 2] += new_w/2 - image_center[0]
M[1, 2] += new_h/2 - image_center[1]

dst = cv.warpAffine(img, M, (new_w, new_h))

cv.imshow('rotated_image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

# Question 4

In [31]:
img1 = cv.imread('native-bee.png')
img2 = cv.imread('flower.jfif')

# Convert to GRAY
img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
resized_img2 = np.zeros((img1.shape[0], img1.shape[1]), dtype = np.uint8)
resized_img2[35:120, 85:175] = img2gray[35:120, 85:175]

# Invert mask
ret, mask = cv.threshold(resized_img2, 70, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask) 

# Convert to BGR and paste it into bee image
resized_img2_rgb = cv.cvtColor(resized_img2, cv.COLOR_GRAY2BGR)
resized_img2_rgb[35:120, 85:175] = img2[35:120, 85:175]

ROI = cv.bitwise_and(img1, img1, mask = mask_inv)

flower = cv.bitwise_and(resized_img2_rgb, resized_img2_rgb, mask = mask)

# Place flower in ROI
dst = cv.add(ROI, flower)

cv.imshow('flower_bee', dst)
cv.waitKey(0)
cv.destroyAllWindows()